In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
from sklearn.preprocessing import StandardScaler

# models (linear regression, krr, xgb, tabPFN)
from sklearn.linear_model import LinearRegression
from sklearn.kernel_ridge import KernelRidge #krr
import xgboost as xgb #xgb

import warnings
warnings.simplefilter('ignore')

#### <b>Load all data and run baseline models</b>

In [2]:
data_feat = pd.read_csv("RDKit_topological.csv")
data_feat = data_feat.dropna(axis = 1)
data_train = pd.read_csv("train.csv")

descriptor_names = data_feat.columns[2::].tolist()
labels = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
data_feat = data_feat[['SMILES'] + descriptor_names]
data_train = data_train[['SMILES'] + labels]

data_concat = data_feat.merge(data_train, on = 'SMILES', how = 'inner')
data_concat.head()

,SMILES,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,...,spectral_radius,number_of_cycles,number_of_stars,diameter,number_of_branches,Tg,FFV,Tc,Density,Rg
0,*CC(*)c1ccccc1C(=O)OCCCCCC,12.144536,12.144536,0.105927,-0.105927,0.500278,13.705882,232.323,212.163,232.146330,...,2.347213,1,2,12,4,NaN,0.374645,0.205667,NaN,NaN
1,*Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...,3.523412,3.523412,0.098918,0.098918,0.125364,16.777778,598.919,544.487,598.428700,...,2.512983,5,2,22,13,NaN,0.370410,NaN,NaN,NaN
2,*Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...,13.714745,13.714745,0.107441,-3.829434,0.092387,16.301370,1003.207,952.807,1002.289625,...,2.494566,10,2,45,25,NaN,0.378860,NaN,NaN,NaN
3,*Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...,3.978671,3.978671,0.054569,-0.202102,0.209590,11.523810,542.726,508.454,542.272199,...,2.598093,6,2,13,16,NaN,0.387324,NaN,NaN,NaN
4,*Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...,13.703218,13.703218,0.068062,-0.686332,0.014164,15.885714,965.154,896.610,964.483374,...,2.425533,6,2,43,18,NaN,0.355470,NaN,NaN,NaN


#### <b>Linear regression</b>

In [4]:
# for linear regression w/o normalization
print("WITHOUT NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]]
    data_concat_prime = data_concat_prime.dropna()
    X, y = data_concat_prime[descriptor_names], data_concat_prime[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    reg = LinearRegression().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

# for linear regression w/ normalization
print()
print("WITH NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]].dropna()
    X = data_concat_prime[descriptor_names]
    y = data_concat_prime[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    #normalization (because might be sensitive)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    reg = LinearRegression().fit(X_train_scaled, y_train)
    y_pred = reg.predict(X_test_scaled)

    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

WITHOUT NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: -0.268, MAE: 96.442
For FFV and with 7030 available datapoints, SRCC: nan, MAE: 0.021
For Tc and with 737 available datapoints, SRCC: 0.857, MAE: 0.032
For Density and with 613 available datapoints, SRCC: 0.400, MAE: 0.103
For Rg and with 614 available datapoints, SRCC: 0.274, MAE: 3.415

WITH NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: 0.660, MAE: 72.072
For FFV and with 7030 available datapoints, SRCC: 0.885, MAE: 4.566
For Tc and with 737 available datapoints, SRCC: 0.879, MAE: 0.031
For Density and with 613 available datapoints, SRCC: 0.815, MAE: 19457889.937
For Rg and with 614 available datapoints, SRCC: 0.754, MAE: 2.597


Couple of things:
- What the.. Why is density's MAE so high but has a healthy SRCC LOL
- Linear (and probably Kernel) regression are super sensitive to scales for features (as expected, but not to this degree)
- When looking at the normalization results for linear regression, it can be shown that all labels apart from Tg, density and Rg have really good performances i.e. they are captured well with a linear regression (there are most likely variables that linearly correlate heavily with these labels - check EDA)
- let us try using a nonlinear model.. we can use quadratic, but the feature space will become gigantic (N + N + N(N-1)/2), and we already have a good amount of features. let's try KRR

#### <b>Kernel Ridge Regression</b>

In [5]:
# w/o normalization
print("WITHOUT NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]]
    data_concat_prime = data_concat_prime.dropna()
    X, y = data_concat_prime[descriptor_names], data_concat_prime[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    reg = KernelRidge().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

# w/ normalization
print()
print("WITH NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]].dropna()
    X = data_concat_prime[descriptor_names]
    y = data_concat_prime[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    #normalization (because might be sensitive)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    reg = KernelRidge().fit(X_train_scaled, y_train)
    y_pred = reg.predict(X_test_scaled)

    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

WITHOUT NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: 0.399, MAE: 116.780
For FFV and with 7030 available datapoints, SRCC: 0.215, MAE: 0.367
For Tc and with 737 available datapoints, SRCC: 0.515, MAE: 0.251
For Density and with 613 available datapoints, SRCC: 0.131, MAE: 1.015
For Rg and with 614 available datapoints, SRCC: 0.227, MAE: 15.985

WITH NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: 0.665, MAE: 100.042
For FFV and with 7030 available datapoints, SRCC: 0.882, MAE: 0.367
For Tc and with 737 available datapoints, SRCC: 0.886, MAE: 0.250
For Density and with 613 available datapoints, SRCC: 0.848, MAE: 0.953
For Rg and with 614 available datapoints, SRCC: 0.825, MAE: 16.412


Couple of comments:
- FFV, Tc, density predictions improved and are, IMO, quite sufficient right now
- Tg makes me quite nervous - it barely improved from linear regression; maybe representation is poor (future work)
- This doesn't mean that we should count KRR out immediately; these are baseline model hyperparameters.. Gamma will play a relatively large role in this (we should tune both KRR and upcoming models like XGB)
- KRR also struggles with variable-to-variable interactions; recall from EDA that there are many weakly correlated features with Tg - perhaps XGB can remedy this?
- KRR may also struggle because we're working without feature selection for baseline models.. another reason why to do selection --> hyp opt. :)

#### <b>XGBoost</b>

In [3]:
# w/o normalization
print("WITHOUT NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]]
    data_concat_prime = data_concat_prime.dropna()
    Q1 = data_concat_prime[label].quantile(0.25)
    Q3 = data_concat_prime[label].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    data_concat_prime = data_concat_prime[(data_concat_prime[label] >= lower) & (data_concat_prime[label] <= upper)]
    data_concat_prime = data_concat_prime.clip(upper = 1e6)
    
    X, y = data_concat_prime[descriptor_names], data_concat_prime[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    reg = xgb.XGBRegressor().fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

# w/ normalization
print()
print("WITH NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]].dropna()
    Q1 = data_concat_prime[label].quantile(0.25)
    Q3 = data_concat_prime[label].quantile(0.75)
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    data_concat_prime = data_concat_prime[(data_concat_prime[label] >= lower) & (data_concat_prime[label] <= upper)]
    data_concat_prime = data_concat_prime.clip(upper = 1e6)
    X = data_concat_prime[descriptor_names]
    y = data_concat_prime[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    #normalization (because might be sensitive)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    reg = xgb.XGBRegressor().fit(X_train_scaled, y_train)
    y_pred = reg.predict(X_test_scaled)

    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

WITHOUT NORMALIZATION:
For Tg and with 498 available datapoints, SRCC: 0.703, MAE: 59.021
For FFV and with 6761 available datapoints, SRCC: 0.928, MAE: 0.006
For Tc and with 737 available datapoints, SRCC: 0.889, MAE: 0.028
For Density and with 592 available datapoints, SRCC: 0.947, MAE: 0.023
For Rg and with 612 available datapoints, SRCC: 0.756, MAE: 2.081

WITH NORMALIZATION:
For Tg and with 498 available datapoints, SRCC: 0.703, MAE: 59.021
For FFV and with 6761 available datapoints, SRCC: 0.927, MAE: 0.006
For Tc and with 737 available datapoints, SRCC: 0.892, MAE: 0.027
For Density and with 592 available datapoints, SRCC: 0.947, MAE: 0.023
For Rg and with 612 available datapoints, SRCC: 0.755, MAE: 2.081


#### <b>We can try TabPFN; shown to work on small data</b>

In [9]:
from tabpfn import TabPFNRegressor

In [10]:
# w/o normalization
print("WITHOUT NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]]
    data_concat_prime = data_concat_prime.dropna()
    data_concat_prime = data_concat_prime.clip(upper = 1e6)
    X, y = data_concat_prime[descriptor_names], data_concat_prime[label]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    reg = TabPFNRegressor() #pre-trained on synthetic data, apparently
    #reg = TabPFNRegressor.create_default_for_version(ModelVersion.V2)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

# w/ normalization
print()
print("WITH NORMALIZATION:")
for label in labels:
    data_concat_prime = data_concat[descriptor_names + [label]].dropna()
    data_concat_prime = data_concat_prime.clip(upper = 1e6)
    X = data_concat_prime[descriptor_names]
    y = data_concat_prime[label]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    #normalization (because might be sensitive)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled  = scaler.transform(X_test)
    reg = TabPFNRegressor() #pre-trained on synthetic data, apparently
    #reg = TabPFNRegressor.create_default_for_version(ModelVersion.V2)
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test_scaled)

    print(f"For {label} and with {len(X)} available datapoints, SRCC: {spearmanr(y_test, y_pred)[0]:.3f}, MAE: {mean_absolute_error(y_test, y_pred):.3f}")

WITHOUT NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: 0.751, MAE: 52.313
For FFV and with 7030 available datapoints, SRCC: 0.966, MAE: 0.005
For Tc and with 737 available datapoints, SRCC: 0.909, MAE: 0.026
For Density and with 613 available datapoints, SRCC: 0.916, MAE: 0.031
For Rg and with 614 available datapoints, SRCC: 0.868, MAE: 1.608

WITH NORMALIZATION:
For Tg and with 511 available datapoints, SRCC: 0.718, MAE: 77.988
For FFV and with 7030 available datapoints, SRCC: 0.319, MAE: 0.083
For Tc and with 737 available datapoints, SRCC: 0.778, MAE: 0.122
For Density and with 613 available datapoints, SRCC: 0.569, MAE: 0.174
For Rg and with 614 available datapoints, SRCC: 0.162, MAE: 8.326
